In [1]:
# !pip install -qq diffusers["torch"] transformers compel wandb

In [2]:
import torch
import wandb
from diffusers import DiffusionPipeline, EulerDiscreteScheduler
from compel import Compel, ReturnedEmbeddingsType

In [3]:
wandb.init(project="stable-diffusion-xl", entity="mratanusarkar", job_type="text-to-image-compel", save_code=True)

config = wandb.config
config.stable_diffusion_checkpoint = "stabilityai/stable-diffusion-xl-base-1.0"
config.refiner_checkpoint = "stabilityai/stable-diffusion-xl-refiner-1.0"
config.offload_to_cpu = False
config.compile_model = False
config.prompt_1 = "a diagram showing flowchart of org chart, chart, diagram---------"
config.prompt_2 = "a rectangular box with 'CEO' written in it, another 2 boxes with 'president' and 'CAO' written in it connected to first box with arrows"
config.negative_prompt_1 = "paiting, nature, static, sd character, low quality, low resolution, greyscale, monochrome, cropped, lowres, realistic, semi-realistic, worst quality, posing"
config.negative_prompt_2 = "low-quality, photograph, painting"
config.seed = 42
config.use_ensemble_of_experts = False
config.num_inference_steps = 100
config.num_refinement_steps = 150
config.high_noise_fraction = 0.8 # Set explicitly only if config.use_ensemble_of_experts is True
config.scheduler_kwargs = {
    "beta_end": 0.012,
    "beta_schedule": "scaled_linear", # one of ["linear", "scaled_linear"]
    "beta_start": 0.00085,
    "interpolation_type": "linear", # one of ["linear", "log_linear"]
    "num_train_timesteps": 1000,
    "prediction_type": "epsilon", # one of ["epsilon", "sample", "v_prediction"]
    "steps_offset": 1,
    "timestep_spacing": "leading", # one of ["linspace", "leading"]
    "trained_betas": None,
    "use_karras_sigmas": False,
}
config.prompt_credits = ""

In [4]:
if config.seed is not None:
    generator = [torch.Generator(device="cuda").manual_seed(config.seed)]
else:
    generator = [torch.Generator(device="cuda")]

In [5]:
pipe = DiffusionPipeline.from_pretrained(
    config.stable_diffusion_checkpoint,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    scheduler=EulerDiscreteScheduler(**config.scheduler_kwargs),
)

if config.offload_to_cpu:
    pipe.enable_model_cpu_offload()
else:
    pipe.to("cuda")

if config.compile_model:
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

In [6]:
if config.prompt_2 == "" and config.negative_prompt_2 == "":
    base_compel = Compel(
        tokenizer=[pipe.tokenizer, pipe.tokenizer_2],
        text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
        returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
        requires_pooled=[False, True]
    )

    base_positive_prompt_embeds, base_positive_prompt_pooled = base_compel(config.prompt)
    base_negative_prompt_embeds, base_negative_prompt_pooled = base_compel(config.negative_prompt)
    base_positive_prompt_embeds, base_negative_prompt_embeds = base_compel.pad_conditioning_tensors_to_same_length([
        base_positive_prompt_embeds, base_negative_prompt_embeds
    ])
else:
    base_compel_1 = Compel(
        tokenizer=pipe.tokenizer,
        text_encoder=pipe.text_encoder,
        returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
        requires_pooled=False,
    )

    base_positive_prompt_embeds_1 = base_compel_1(config.prompt_1)
    base_negative_prompt_embeds_1 = base_compel_1(config.negative_prompt_1)

    base_compel_2 = Compel(
        tokenizer=pipe.tokenizer_2,
        text_encoder=pipe.text_encoder_2,
        returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
        requires_pooled=True,
    )

    base_positive_prompt_embeds_2, base_positive_prompt_pooled = base_compel_2(config.prompt_2)
    base_negative_prompt_embeds_2, base_negative_prompt_pooled = base_compel_2(config.negative_prompt_2)

    (
        base_positive_prompt_embeds_2, base_negative_prompt_embeds_2
    ) = base_compel_2.pad_conditioning_tensors_to_same_length([
        base_positive_prompt_embeds_2, base_negative_prompt_embeds_2
    ])

    base_positive_prompt_embeds = torch.cat((base_positive_prompt_embeds_1, base_positive_prompt_embeds_2), dim=-1)
    base_negative_prompt_embeds = torch.cat((base_negative_prompt_embeds_1, base_negative_prompt_embeds_2), dim=-1)

In [7]:
image = pipe(
    prompt_embeds=base_positive_prompt_embeds,
    pooled_prompt_embeds=base_positive_prompt_pooled,
    negative_prompt_embeds=base_negative_prompt_embeds,
    negative_pooled_prompt_embeds=base_negative_prompt_pooled,
    output_type="pil",
    num_inference_steps=config.num_inference_steps,
    generator=generator,
).images[0]

In [8]:
table = wandb.Table(columns=[
    "Prompt-1",
    "Prompt-2",
    "Negative-Prompt-1",
    "Negative-Prompt-2",
    "Generated-Image"
])

image = wandb.Image(image)

table.add_data(
    config.prompt_1,
    config.prompt_2,
    config.negative_prompt_1,
    config.negative_prompt_2,
    image,
)
wandb.log({
    "Generated-Image": image,
    "Text-to-Image": table
})
wandb.finish()

In [9]:
# !pip install -qq diffusers["torch"] transformers compel wandb

In [10]:
import torch
import wandb
from diffusers import DiffusionPipeline, EulerDiscreteScheduler
from compel import Compel, ReturnedEmbeddingsType

In [11]:
wandb.init(project="stable-diffusion-xl", entity="mratanusarkar", job_type="text-to-image-compel", save_code=True)

config = wandb.config
config.stable_diffusion_checkpoint = "stabilityai/stable-diffusion-xl-base-1.0"
config.refiner_checkpoint = "stabilityai/stable-diffusion-xl-refiner-1.0"
config.offload_to_cpu = False
config.compile_model = False
config.prompt_1 = "a diagram showing flowchart of org chart, chart, diagram---------"
config.prompt_2 = "a rectangular box with 'CEO' written in it, another 2 boxes with 'president' and 'CAO' written in it connected to first box with arrows"
config.negative_prompt_1 = "paiting, nature, static, sd character, low quality, low resolution, greyscale, monochrome, cropped, lowres, realistic, semi-realistic, worst quality, posing"
config.negative_prompt_2 = "low-quality, photograph, painting"
config.seed = 42
config.use_ensemble_of_experts = False
config.num_inference_steps = 100
config.num_refinement_steps = 150
config.high_noise_fraction = 0.8 # Set explicitly only if config.use_ensemble_of_experts is True
config.scheduler_kwargs = {
    "beta_end": 0.012,
    "beta_schedule": "scaled_linear", # one of ["linear", "scaled_linear"]
    "beta_start": 0.00085,
    "interpolation_type": "linear", # one of ["linear", "log_linear"]
    "num_train_timesteps": 1000,
    "prediction_type": "epsilon", # one of ["epsilon", "sample", "v_prediction"]
    "steps_offset": 1,
    "timestep_spacing": "leading", # one of ["linspace", "leading"]
    "trained_betas": None,
    "use_karras_sigmas": False,
}
config.prompt_credits = ""

In [12]:
if config.seed is not None:
    generator = [torch.Generator(device="cuda").manual_seed(config.seed)]
else:
    generator = [torch.Generator(device="cuda")]

In [13]:
pipe = DiffusionPipeline.from_pretrained(
    config.stable_diffusion_checkpoint,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    scheduler=EulerDiscreteScheduler(**config.scheduler_kwargs),
)

if config.offload_to_cpu:
    pipe.enable_model_cpu_offload()
else:
    pipe.to("cuda")

if config.compile_model:
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [14]:
if config.prompt_2 == "" and config.negative_prompt_2 == "":
    base_compel = Compel(
        tokenizer=[pipe.tokenizer, pipe.tokenizer_2],
        text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
        returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
        requires_pooled=[False, True]
    )

    base_positive_prompt_embeds, base_positive_prompt_pooled = base_compel(config.prompt)
    base_negative_prompt_embeds, base_negative_prompt_pooled = base_compel(config.negative_prompt)
    base_positive_prompt_embeds, base_negative_prompt_embeds = base_compel.pad_conditioning_tensors_to_same_length([
        base_positive_prompt_embeds, base_negative_prompt_embeds
    ])
else:
    base_compel_1 = Compel(
        tokenizer=pipe.tokenizer,
        text_encoder=pipe.text_encoder,
        returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
        requires_pooled=False,
    )

    base_positive_prompt_embeds_1 = base_compel_1(config.prompt_1)
    base_negative_prompt_embeds_1 = base_compel_1(config.negative_prompt_1)

    base_compel_2 = Compel(
        tokenizer=pipe.tokenizer_2,
        text_encoder=pipe.text_encoder_2,
        returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
        requires_pooled=True,
    )

    base_positive_prompt_embeds_2, base_positive_prompt_pooled = base_compel_2(config.prompt_2)
    base_negative_prompt_embeds_2, base_negative_prompt_pooled = base_compel_2(config.negative_prompt_2)

    (
        base_positive_prompt_embeds_2, base_negative_prompt_embeds_2
    ) = base_compel_2.pad_conditioning_tensors_to_same_length([
        base_positive_prompt_embeds_2, base_negative_prompt_embeds_2
    ])

    base_positive_prompt_embeds = torch.cat((base_positive_prompt_embeds_1, base_positive_prompt_embeds_2), dim=-1)
    base_negative_prompt_embeds = torch.cat((base_negative_prompt_embeds_1, base_negative_prompt_embeds_2), dim=-1)

In [15]:
image = pipe(
    prompt_embeds=base_positive_prompt_embeds,
    pooled_prompt_embeds=base_positive_prompt_pooled,
    negative_prompt_embeds=base_negative_prompt_embeds,
    negative_pooled_prompt_embeds=base_negative_prompt_pooled,
    output_type="pil",
    num_inference_steps=config.num_inference_steps,
    generator=generator,
).images[0]

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
table = wandb.Table(columns=[
    "Prompt-1",
    "Prompt-2",
    "Negative-Prompt-1",
    "Negative-Prompt-2",
    "Generated-Image"
])

image = wandb.Image(image)

table.add_data(
    config.prompt_1,
    config.prompt_2,
    config.negative_prompt_1,
    config.negative_prompt_2,
    image,
)
wandb.log({
    "Generated-Image": image,
    "Text-to-Image": table
})
wandb.finish()